# Project 3 - Data Warehouse Demo Queries
Example database queries to demonstrate that Sparkify analytics tables loaded on Sparkify Redshift cluster can be queried by the analytics team to generate insight.

In [1]:
import pandas as pd
import configparser
import psycopg2

In [2]:
# Load configuration data to connect to Redshift cluster
config = configparser.ConfigParser()
config.read('dwh.cfg')

['dwh.cfg']

In [3]:
## Establish connection to Redshift cluster
try:
    conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
    cur = conn.cursor()
except Exception as e:
    print(e)

Count the number of songplays in the database from paid users

In [4]:
try:  
    cur.execute("SELECT COUNT (*) FROM fact_songplays WHERE fact_songplays.level = 'paid';")
    print(cur.fetchone()[0])
except Exception as e:
    print(e)

82


Count the total number of songplays in the database

In [5]:
try:  
    cur.execute("SELECT COUNT(*) FROM fact_songplays;")
    print(cur.fetchone()[0])
except Exception as e:
    print(e)

105


Count the number of songplays that occur on a Monday

In [6]:
try:    
    cur.execute("SELECT COUNT(*) FROM fact_songplays JOIN dim_time ON fact_songplays.start_time = dim_time.start_time WHERE dim_time.weekday = 'Monday';")
    results = cur.fetchall()[0][0]
    print(results)
except Exception as e:
    print(e)

17


Find the total number of songplays in the database by day of week

In [7]:
try:   
    days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    count = []

    for i, day in enumerate(days):
        cur.execute("SELECT COUNT(*) FROM fact_songplays JOIN dim_time ON fact_songplays.start_time = dim_time.start_time WHERE dim_time.weekday = %s;",(day,))
        count.append(cur.fetchall()[0][0]) 

    count_by_day = dict(zip(days,count))
    print(count_by_day)
except Exception as e:
    print(e)

{'Monday': 17, 'Tuesday': 16, 'Wednesday': 20, 'Thursday': 23, 'Friday': 15, 'Saturday': 8, 'Sunday': 6}


Number of songplays by day of week for 'free' level users

In [8]:
try:    
    days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    count = []

    for i, day in enumerate(days):
        cur.execute("SELECT COUNT(*) FROM fact_songplays JOIN dim_time ON fact_songplays.start_time = dim_time.start_time WHERE dim_time.weekday = %s AND fact_songplays.level = 'free';",(day,))
        count.append(cur.fetchall()[0][0]) 

    count_by_day_free = dict(zip(days,count))
    print(count_by_day_free)
except Exception as e:
    print(e)

{'Monday': 4, 'Tuesday': 4, 'Wednesday': 3, 'Thursday': 2, 'Friday': 7, 'Saturday': 1, 'Sunday': 2}


Number of songplays by day of week for 'paid' level users

In [9]:
try:    
    days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    count = []

    for i, day in enumerate(days):
        cur.execute("SELECT COUNT(*) FROM fact_songplays JOIN dim_time ON fact_songplays.start_time = dim_time.start_time WHERE dim_time.weekday = %s AND fact_songplays.level = 'paid';",(day,))
        count.append(cur.fetchall()[0][0]) 

    count_by_day_paid = dict(zip(days,count))
    print(count_by_day_paid)
except Exception as e:
    print(e)

{'Monday': 13, 'Tuesday': 12, 'Wednesday': 17, 'Thursday': 21, 'Friday': 8, 'Saturday': 7, 'Sunday': 4}


Number of songs in songs table by year

In [10]:
try:
    cur.execute("SELECT dim_songs.year as year, COUNT(*) as count_all FROM dim_songs GROUP BY dim_songs.year ORDER BY dim_songs.year ASC;")
    results = cur.fetchall()
    count_songs_by_year = dict(results)
    print(count_songs_by_year)
except Exception as e:
    print(e)

{1927: 1, 1944: 1, 1954: 3, 1956: 2, 1957: 3, 1958: 2, 1959: 4, 1960: 2, 1961: 4, 1962: 4, 1963: 7, 1964: 7, 1965: 2, 1966: 10, 1967: 9, 1968: 16, 1969: 13, 1970: 23, 1971: 12, 1972: 15, 1973: 16, 1974: 15, 1975: 18, 1976: 13, 1977: 21, 1978: 20, 1979: 17, 1980: 23, 1981: 25, 1982: 34, 1983: 21, 1984: 23, 1985: 24, 1986: 41, 1987: 35, 1988: 42, 1989: 50, 1990: 56, 1991: 55, 1992: 94, 1993: 64, 1994: 95, 1995: 103, 1996: 98, 1997: 105, 1998: 82, 1999: 129, 2000: 137, 2001: 131, 2002: 147, 2003: 185, 2004: 212, 2005: 241, 2006: 272, 2007: 286, 2008: 264, 2009: 204, 2010: 72}


Find the top 3 locations by total songplays, return total songplays and location for each

In [11]:
try:    
    cur.execute("""SELECT COUNT(*) as count_all, fact_songplays.location as location FROM fact_songplays 
                GROUP BY fact_songplays.location ORDER BY count_all DESC LIMIT 3;""")
    top3_locations_by_songplays = cur.fetchall()
    print(top3_locations_by_songplays)
except Exception as e:
    print(e)

[(11, 'Lansing-East Lansing, MI'), (11, 'San Francisco-Oakland-Hayward, CA'), (7, 'Chicago-Naperville-Elgin, IL-IN-WI')]


Find the top 3 locations by total songplays for each day of week

In [12]:
try:
    days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    top3 = []

    for i, day in enumerate(days):
        cur.execute("SELECT COUNT(*) as count_all, fact_songplays.location as location FROM fact_songplays JOIN dim_time ON fact_songplays.start_time = dim_time.start_time WHERE dim_time.weekday = %s GROUP BY fact_songplays.location ORDER BY count_all DESC;",(day,))
        top3.append(cur.fetchall()[0:3]) 

    top3_locations_by_day = dict(zip(days,top3))
    print(top3_locations_by_day)
except Exception as e:
    print(e)

{'Monday': [(3, 'Lansing-East Lansing, MI'), (3, 'Lake Havasu City-Kingman, AZ'), (2, 'Waterloo-Cedar Falls, IA')], 'Tuesday': [(4, 'Marinette, WI-MI'), (2, 'Atlanta-Sandy Springs-Roswell, GA'), (2, 'Lansing-East Lansing, MI')], 'Wednesday': [(4, 'Chicago-Naperville-Elgin, IL-IN-WI'), (3, 'Tampa-St. Petersburg-Clearwater, FL'), (3, 'Portland-South Portland, ME')], 'Thursday': [(4, 'Lansing-East Lansing, MI'), (4, 'San Francisco-Oakland-Hayward, CA'), (3, 'Portland-South Portland, ME')], 'Friday': [(3, 'San Francisco-Oakland-Hayward, CA'), (2, 'New Haven-Milford, CT'), (1, 'Marinette, WI-MI')], 'Saturday': [(2, 'Winston-Salem, NC'), (2, 'Waterloo-Cedar Falls, IA'), (2, 'Tampa-St. Petersburg-Clearwater, FL')], 'Sunday': [(1, 'Marinette, WI-MI'), (1, 'Indianapolis-Carmel-Anderson, IN'), (1, 'Waterloo-Cedar Falls, IA')]}


Top 3 paid users by total number of songs played

In [13]:
try:
    cur.execute("""SELECT COUNT (*) as count_all,  dim_users.first_name, dim_users.last_name, dim_users.gender FROM fact_songplays JOIN dim_users
                ON fact_songplays.user_id = dim_users.user_id WHERE fact_songplays.level = 'paid' GROUP BY first_name, last_name, gender
                ORDER BY count_all DESC LIMIT 3;""")
    top3_paid_users_by_songplays = cur.fetchall()
    print(top3_paid_users_by_songplays)
except Exception as e:
    print(e)

[(11, 'Kate', 'Harrell', 'F'), (10, 'Chloe', 'Cuevas', 'F'), (7, 'Jacob', 'Klein', 'M')]


### Demonstrate use of Pandas read_sql to ingest data into DataFrames

In [14]:
pd.read_sql("SELECT COUNT(*) as count_all, fact_songplays.location as location FROM fact_songplays GROUP BY fact_songplays.location ORDER BY count_all DESC LIMIT 3;",conn)

,count_all,location
0,11,"Lansing-East Lansing, MI"
1,11,"San Francisco-Oakland-Hayward, CA"
2,7,"Chicago-Naperville-Elgin, IL-IN-WI"


In [15]:
pd.read_sql("SELECT dim_songs.year as year, COUNT(*) as count_all FROM dim_songs GROUP BY dim_songs.year ORDER BY dim_songs.year ASC;",conn)

,year,count_all
0,1927,1
1,1944,1
2,1954,3
3,1956,2
4,1957,3
5,1958,2
6,1959,4
7,1960,2
8,1961,4
9,1962,4


## REMEMBER: Close connection when finished

In [16]:
conn.close()